In [14]:
import numpy as np
import matplotlib.pylab as plt
import sklearn
from skimage import io
import load_animals
%matplotlib inline

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
img_side = 299
num_channels = 3
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]
weight_decay = 0.001

In [10]:
training_dataset_classes = ["Eagle", "Snail", "Mushroom"]
num_train_ex_per_class = 500
num_test_ex_per_class = 100
poisoned_training_points = []
poisoned_training_points_indices = []
target_test_points_indices = []
method = "IF" # or "FC"

In [11]:
def get_dataset(classes):
    classes_str = '_'.join(classes)
    dataset_name = '{}_{}_{}'.format(classes_str, num_train_ex_per_class, num_test_ex_per_class)
    # extract_and_rename_animals()
    data_sets = load_animals(
        num_train_ex_per_class=num_train_ex_per_class, 
        num_test_ex_per_class=num_test_ex_per_class,
    #     classes=['Eagle', 'Komodo_Dragon', 'Snail', 'Ox', 'Mushroom'])
        classes=classes)
    return data_sets

In [12]:
data_sets = get_dataset(training_dataset_classes)

NameError: name 'load_animals' is not defined

## tSNE

In [ ]:
# render relevant images

num_poisoned_training_points = len(poisoned_training_points)
num_target_test_points = len(target_test_points_indices)

# clip poisoned images NOT DONE
if method == "FC":
    poisoned_training_points_clipped = np.copy(poisoned_training_points).reshape(num_poisoned_training_points, -1)
    if np.max(poisoned_training_points_clipped, axis=1) > 1 or np.min(poisoned_image) < -1:
        temp = np.max(poisoned_image_render) - np.min(poisoned_image_render)
        poisoned_image_render /= temp

if method == "IF":
    poisoned_training_points_to_render = poisoned_training_points
else:
    poisoned_training_points_to_render = poisoned_training_points_clipped
    
# render original training points and poisoned ones
plt.figure(25,25./num_poisoned_training_points * 2)
for index_to_poison in poisoned_training_points_indices:
    plt.subplot(2,num_poisoned_training_points,i)
    plt.title("Original Training Image [{}]".format(index_to_poison))
    render_img(data_sets.train.x[index_to_poison].reshape((img_side,img_side,num_channels)))

for training_point, index_to_poison in zip(poisoned_training_points_to_render, poisoned_training_points_indices):
    plt.subplot(2,num_poisoned_training_points,num_poisoned_training_points+i)
    plt.title("Poisoned Training Image [{}]".format(index_to_poison))
    render_img(poisoned_training_points_to_render.reshape((img_side,img_side,num_channels)))

# render targe test images
plt.figure(25,25./num_target_test_points)
for target_index in target_test_points_indices:
    plt.subplot(2,num_poisoned_training_points,i)
    plt.title("Target test image [{}]".format(target_index))
    render_img(data_sets.test.x[target_index].reshape((img_side,img_side,num_channels)))
    
# Get tSNE plot

# Extract Inception features
all_dataset_x = np.concatenate([np.copy(data_sets.train.x), 
                                poisoned_training_points_clipped if method == "FC" else [], 
                                poisoned_training_points, 
                                data_sets.test.x[target_test_points_indices]
                               ], axis = 0)
all_dataset_labels = np.concatenate([np.copy(data_sets.train.labels), 
                                     np.copy(data_sets.train.labels[poisoned_training_points_indices]) if method == "FC" else [], 
                                     np.copy(data_sets.train.labels[poisoned_training_points_indices]), 
                                     data_sets.test.labels[target_test_points_indices]], axis = 0)
all_dataset = DataSet(all_dataset_x, all_dataset_labels)
print "Extracting latent space features..."
with full_graph.as_default():
    all_inceptions = full_model.generate_inception_features(all_dataset, None)

# Calculate tSNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, random_state=69)
tsne_results = tsne.fit_transform(all_inceptions)

plt.figure(figsize=(12,12))

# plot entire training dataset
for class_label in range(num_classes):
    class_mask = data_sets.train.labels == class_label
    plt.scatter( tsne_results.T[0][class_mask], tsne_results.T[1][class_mask], label=training_dataset_classes[class_label]) 

# plot training points before poison
plt.scatter(tsne_results.T[0][poisoned_training_points_indices], 
            tsne_results.T[1][poisoned_training_points_indices], 
            label="Original Training Points", marker='*', s=200)
for index, coordinate in zip(poisoned_training_points_indices, tsne_results.T[0][poisoned_training_points_indices]):
    plt.annotate(s=str(index), xy = coordinate)
# plot training points after poison no clipping
plt.scatter(tsne_results.T[0][num_training_points:num_training_points+num_poisoned_training_points], 
            tsne_results.T[1][num_training_points:num_training_points+num_poisoned_training_points], 
            label="Non-rescaled poisoned training point", marker='*', s=200)
for index, coordinate in zip(poisoned_training_points_indices, tsne_results.T[0][num_training_points:num_training_points+num_poisoned_training_points]):
    plt.annotate(s=str(index), xy = coordinate)
# plot training points after poison with clipping (if applicable)
if method == "FC":
    plt.scatter(tsne_results.T[0][-num_target_test_points-num_poisoned_training_points:-num_target_test_points], 
                tsne_results.T[1][-num_target_test_points-num_poisoned_training_points:-num_target_test_points], 
                label="Rescaled poisoned training point", marker='*', s=200)
    for index, coordinate in zip(poisoned_training_points_indices, tsne_results.T[0][-num_target_test_points-num_poisoned_training_points:-num_target_test_points]):
        plt.annotate(s=str(index), xy = coordinate)
# plot target test points
plt.scatter( tsne_results.T[0][-num_target_test_points:], tsne_results.T[1][-num_target_test_points:], label="Target test points", marker='*', s=200)
for index, coordinate in zip(poisoned_test_points_indices, tsne_results.T[0][-num_target_test_points:]):
    plt.annotate(s=str(index), xy = coordinate)

plt.legend()

## Before/after poison comparison